In [1]:
# general purpose python
import collections
import datetime
import glob
import importlib
import itertools
import json
import math
import os
import pickle
import random
import re
import shutil
import sys
import time
import warnings

# general purpose data science
import IPython
import ipywidgets as ipw
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs
import pylab
import scipy
import seaborn as sns
import sklearn
from sklearn import *
import statsmodels as sm

# computer vision
import cv2
import imageio
import PIL
from PIL import *

# deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

# geospatial
import rasterio as rio
import rasterio.features

warnings.filterwarnings('ignore')

np.random.seed(1337)

mpl.rcParams['figure.dpi'] = 400

IPython.core.display.display(IPython.core.display.HTML("<style>.container { width:100% !important; }</style>"))

pd.options.display.max_colwidth = 32
pd.options.display.float_format = '{:,.6f}'.format
pd.options.display.expand_frame_repr = False

%matplotlib inline

sns.set(font_scale=1.3)
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

plotly.offline.init_notebook_mode(connected=True)
plotly.io.templates.default = 'plotly_white'

In [2]:
data_dir = '../../data/movielens/'

## Load the data

In [3]:
movies = pd.read_csv(f'{data_dir}/movies_clean.csv')
ratings = pd.read_csv(f'{data_dir}/ratings_train.csv')
my_ratings = pd.read_csv(f'{data_dir}/movielens-ratings.csv') # https://movielens.org/profile/settings/import-export

## Find a set of popular movies for clustering

In [4]:
# find a set of popular movies that the current user rated to use as a basis for the clustering
top_movies_count = 10
popular_movie_ids = ratings.movie_id.value_counts().head(top_movies_count).index.tolist()
my_movie_ids = my_ratings.movie_id.tolist()
clustering_movie_ids = list(set(popular_movie_ids).intersection(set(my_movie_ids)))
print(len(clustering_movie_ids))
movies[movies.movie_id.isin(clustering_movie_ids)][['movie_id', 'title']]

7


,movie_id,title
257,260,Star Wars: Episode IV - A Ne...
293,296,Pulp Fiction (1994)
315,318,"Shawshank Redemption, The (1..."
476,480,Jurassic Park (1993)
583,589,Terminator 2: Judgment Day (...
587,593,"Silence of the Lambs, The (1..."
2486,2571,"Matrix, The (1999)"


## Find users who rated the same movies

In [5]:
clustering_movie_ratings = ratings[ratings.movie_id.isin(clustering_movie_ids)]
clustering_movie_rating_counts = clustering_movie_ratings.groupby(['user_id']).movie_id.count()
clustering_user_ids = clustering_movie_rating_counts[clustering_movie_rating_counts == len(clustering_movie_ids)].index.tolist()
clustering_ratings = clustering_movie_ratings[clustering_movie_ratings.user_id.isin(clustering_user_ids)]
print('number of users: ', len(clustering_user_ids))
print('number of ratings: ', len(clustering_ratings))

number of users:  2355
number of ratings:  16485


## Prepare the data for clustering

In [6]:
# append the current user's ratings to the ratings data set
my_ratings['user_id'] = -1
my_ratings = my_ratings[my_ratings.movie_id.isin(clustering_movie_ids)][['user_id', 'movie_id', 'rating']]
clustering_ratings = clustering_ratings[['user_id', 'movie_id', 'rating']]
clustering_ratings = pd.concat([clustering_ratings, my_ratings])

In [7]:
# create rectangular ratings matrix
clustering_ratings = clustering_ratings.sort_values(by=['user_id', 'movie_id'])
r = clustering_ratings.pivot(index='user_id', columns='movie_id')
r.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, -1 to 138387
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (rating, 260)   2356 non-null   float64
 1   (rating, 296)   2356 non-null   float64
 2   (rating, 318)   2356 non-null   float64
 3   (rating, 480)   2356 non-null   float64
 4   (rating, 589)   2356 non-null   float64
 5   (rating, 593)   2356 non-null   float64
 6   (rating, 2571)  2356 non-null   float64
dtypes: float64(7)
memory usage: 147.2 KB


In [8]:
r.head(10)

rating                                                      
movie_id     260      296      318      480      589      593      2571
user_id                                                                
-1       4.500000 5.000000 3.000000 4.000000 4.500000 5.000000 5.000000
 90      3.500000 3.500000 4.000000 3.500000 3.000000 3.500000 2.000000
 91      4.500000 3.500000 4.000000 3.500000 2.500000 3.000000 3.500000
 124     3.000000 5.000000 5.000000 2.000000 4.000000 5.000000 5.000000
 208     4.500000 5.000000 4.500000 4.500000 2.500000 4.500000 3.000000
 237     5.000000 5.000000 5.000000 4.500000 4.500000 4.000000 5.000000
 359     5.000000 5.000000 5.000000 4.000000 5.000000 5.000000 3.000000
 546     3.000000 5.000000 4.000000 3.000000 3.500000 3.500000 4.000000
 600     5.000000 1.000000 4.000000 4.000000 5.000000 5.000000 4.000000
 609     4.500000 5.000000 3.500000 4.000000 2.500000 3.000000 5.000000

In [9]:
# normalize ratings (note: this is incorrect, the average of all movie ratings should be used here, but this is good enough for now)
rv = r.values
rv -= (rv.sum(axis=1)[:, np.newaxis] / 12.0)
rv.shape

(2356, 7)

## Clustering and visualization

In [10]:
# run the K-means clustering
n_clusters = 8
kmeans = sklearn.cluster.KMeans(n_clusters=n_clusters).fit(rv)
print('cluster label for the current user: ', kmeans.labels_[0])

cluster label for the current user:  1


In [11]:
# PCA only for visualization to project the data to 3D
pca = sklearn.decomposition.PCA(n_components=3)
rv_pca = pca.fit_transform(rv)
print('explained variane ratio with 3 dimensions: ', sum(pca.explained_variance_ratio_))

explained variane ratio with 3 dimensions:  0.5442367733905352


In [12]:
data = [
    plotly.graph_objs.Scatter3d(
    x=rv_pca[:, 0],
    y=rv_pca[:, 1],
    z=rv_pca[:, 2],
    mode='markers',
    marker={
        'size': 3,
        'color': kmeans.labels_,
        'colorscale': 'Rainbow',
        'line': {'width': 0.5}}),
    plotly.graph_objs.Scatter3d(
    x=[rv_pca[0, 0]],
    y=[rv_pca[0, 1]],
    z=[rv_pca[0, 2]],
    mode='markers',
    marker={
        'size': 10,
        'color': 'black'})]
layout = plotly.graph_objs.Layout(
    autosize=False,
    width=1200,
    height=900,
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},
    showlegend=False)
fig = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, {'xaxis': {'range': [-1, 1]}}, config={'displayModeBar': False})

## Nearest neighbors

In [13]:
neigh = sklearn.neighbors.NearestNeighbors(n_neighbors=11)
neigh.fit(rv)
neighbors = neigh.kneighbors(rv[0:,:])[1][0]
neighbor_ids = []
for neighbor in neighbors:
    neighbor_ids.append(list(r.index)[neighbor])
r[r.index.isin(neighbor_ids)]

rating                                                      
movie_id     260      296      318      480      589      593      2571
user_id                                                                
-1       1.916667 2.416667 0.416667 1.416667 1.916667 2.416667 2.416667
 15814   1.500000 2.500000 0.500000 1.500000 1.500000 2.500000 2.500000
 16118   1.916667 1.916667 0.916667 1.916667 1.916667 1.916667 2.416667
 28933   2.291667 2.291667 0.791667 1.791667 2.291667 2.291667 1.791667
 49865   2.333333 2.333333 0.833333 1.333333 2.333333 1.833333 2.333333
 68932   2.416667 2.416667 0.416667 1.416667 1.416667 2.416667 2.416667
 88785   1.500000 2.000000 1.000000 1.500000 2.000000 2.000000 2.500000
 98412   1.958333 2.458333 0.458333 0.958333 2.458333 1.958333 2.458333
 106476  1.458333 2.458333 0.958333 1.458333 1.458333 2.458333 2.458333
 117716  2.416667 2.416667 0.416667 1.416667 1.416667 2.416667 2.416667
 136063  1.500000 2.500000 1.000000 1.000000 2.000000 2.500000 2.000000